## Make ras2fim/ripple boundary dataset

###### This comes in two parts, it will load all of the stats.csv from the ripple downloads, which are often split into smaller groups. ie) for FIM30, there was 485 MC (model collections), but they were downloaded in sets of 50. 

Last edited: Mar 5, 2025

*** Processing steps
1) Load all of the ripple stats csvs
   
2) load the old ras2fm v2 data. At this point, we don't have a specific dataset for ras2fim that has the huc number and number of models included. We will make a simple csv with columns of HUC and model_count.

3) Merge the ripple and ras2fim df together

4) Make a new dataframe starting with just unique HUCs. Each HUC can iterate back through the original merged df and look for the count of models for ripple mip, ripple ble and ras2fim

5) save as csv and gpkg  (csv for HV and gkpg for easy visual


In [11]:
import os
import glob
import stat

import geopandas as gpd
import pandas as pd


# Display all rows
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_rows', 10)

# Display all columns
pd.set_option('display.max_columns', 10)

# Display full width of columns
pd.set_option('display.max_colwidth', None)

print("Done loading imports")

Done loading imports


In [15]:
# GLOBAL VARIABLES

# *****************************
# NOTE: Careful about checking some of this in if it has actual server names or paths
ROOT_PATH = "/efs-drives/fim-dev-efs"
# *****************************

WBD_HUC8_FILE_PATH = f"{ROOT_PATH}/fim-data/inputs/wbd/WBD_National_HUC8_EPSG_5070_HAND_domain.gpkg"

RIPPLE_STATS_CSV_DIR = f"{ROOT_PATH}/fim-data/ripple/fim_30_prod_data/stats_csv_temp/"
RAS2FIM_STATUS_FILE = f"{ROOT_PATH}/fim-data/ripple/fim_30_prod_data/ras2fim_v2_huc_list_w_feature_counts.csv"
OUTPUT_CSV_path = "/home/rdp-user/outputs/hecras_boundaries.csv"

is_verbose = True

print("Done loading global variables")


Done loading global variables


In [3]:
# Load ripple csv's

stats_files = glob.glob(RIPPLE_STATS_CSV_DIR + "*.csv")
df_ripple_stats_files = []

for filename in stats_files:
    df = pd.read_csv(filename,
                     index_col=None,
                     usecols=['huc', 'source', 'num_features'],
                     dtype={'huc': str})
    df_ripple_stats_files.append(df)

df_ripple_stats = pd.concat(df_ripple_stats_files, ignore_index=True)

if is_verbose:
    print(df_ripple_stats)


    source      huc  num_features
0      ble  8040205            48
1      mip  8040302            51
2      mip  8040303           128
3      mip  8040304            22
4      mip  8060202            92
..     ...      ...           ...
480    mip  8030207           171
481    ble  8040101            75
482    mip  8040101             4
483    ble  8040203           159
484    mip  8040204             1

[485 rows x 3 columns]


In [4]:
# Load the ras2fim data
df_ras2fim = pd.read_csv(RAS2FIM_STATUS_FILE,
                         index_col=None,
                         dtype={'huc': str})
df_ras2fim["source"] = "ras2fim"
if is_verbose:
    print(df_ras2fim)


         huc  num_features   source
0   12010005           947  ras2fim
1   12030101           607  ras2fim
2   12030103           466  ras2fim
3   12030105           247  ras2fim
4   12030106           352  ras2fim
5   12030108           377  ras2fim
6   12030201          1068  ras2fim
7   12030202          1764  ras2fim
8   12040101           619  ras2fim
9   12070102           546  ras2fim
10  12090301           898  ras2fim
11  12100202            57  ras2fim


In [5]:
df_stats = pd.concat([df_ripple_stats, df_ras2fim], ignore_index=True)

df_stats['huc'] = df_stats['huc'].str.zfill(8)

if is_verbose:
    print(df_stats)

      source       huc  num_features
0        ble  08040205            48
1        mip  08040302            51
2        mip  08040303           128
3        mip  08040304            22
4        mip  08060202            92
..       ...       ...           ...
492  ras2fim  12030202          1764
493  ras2fim  12040101           619
494  ras2fim  12070102           546
495  ras2fim  12090301           898
496  ras2fim  12100202            57

[497 rows x 3 columns]


In [6]:
# Group by 'Category' and pivot 'Item' to columns
df_pivot = df_stats.pivot_table(index='huc', columns='source', values='num_features')

df_pivot["ble"].fillna("0", inplace = True)
df_pivot["mip"].fillna("0", inplace = True)
df_pivot["ras2fim"].fillna("0", inplace = True)

df_pivot['ble'] = df_pivot['ble'].astype(int)
df_pivot['mip'] = df_pivot['mip'].astype(int)
df_pivot['ras2fim'] = df_pivot['ras2fim'].astype(int)

# drop blank rows
df_pivot = df_pivot[ ((df_pivot["ble"] > 0) | (df_pivot["mip"] > 0) | (df_pivot["ras2fim"] > 0)) ]

if is_verbose:
    print(df_pivot)

source    ble  mip  ras2fim
huc                        
02040101    0  212        0
02040103    0  208        0
02040104    0  388        0
02040105    0    3        0
02040106    0  419        0
...       ...  ...      ...
17120002    0   23        0
18010201    0   10        0
18010202    0   80        0
18010203    0    5        0
18010204    0   61        0

[435 rows x 3 columns]


In [7]:

# just testing
#df = df_pivot.loc[df_pivot['ble'] > 0]
# df_pivot.loc[(df_pivot['mip'] == 0) & (df_pivot['ble'] > 0) ]
# df_pivot.loc[df_pivot['ras2fim'] > 0]


In [8]:
# find the source with the highest number of features.
cols_to_check = ['ble', 'mip', 'ras2fim']
# df_pivot["selected_source"] = 
df_pivot["selected_source"] = df_pivot[cols_to_check].idxmax(axis=1)

if is_verbose:
    print(df_pivot)


source    ble  mip  ras2fim selected_source
huc                                        
02040101    0  212        0             mip
02040103    0  208        0             mip
02040104    0  388        0             mip
02040105    0    3        0             mip
02040106    0  419        0             mip
...       ...  ...      ...             ...
17120002    0   23        0             mip
18010201    0   10        0             mip
18010202    0   80        0             mip
18010203    0    5        0             mip
18010204    0   61        0             mip

[435 rows x 4 columns]


In [9]:
# table adjustments
df_pivot.rename(columns={"ble": "num_ble_features", "mip": "num_mip_features", "ras2fim": "num_ras2fim_features"}, inplace=True)
df_pivot["is_active"] = "True"
if is_verbose:
    print(df_pivot)

source    num_ble_features  num_mip_features  num_ras2fim_features  \
huc                                                                  
02040101                 0               212                     0   
02040103                 0               208                     0   
02040104                 0               388                     0   
02040105                 0                 3                     0   
02040106                 0               419                     0   
...                    ...               ...                   ...   
17120002                 0                23                     0   
18010201                 0                10                     0   
18010202                 0                80                     0   
18010203                 0                 5                     0   
18010204                 0                61                     0   

source   selected_source is_active  
huc                                 
02040101       

In [19]:
# Add geometries from the HUCs from the WBD

# Load the WBD
wbd = gpd.read_file(WBD_HUC8_FILE_PATH)[["HUC8", "geometry"]]

# merge with my csv  (gpd)
boundaries_df = df_pivot.merge(wbd, left_on='huc', right_on='HUC8')

if is_verbose:
    # print(boundaries_df.head(1))
    print(f"Total Rec count is {len(boundaries_df)}")




Total Rec count is 434


In [ ]:

#df_pivot.to_csv(OUTPUT_CSV_path)
#os.chmod(OUTPUT_CSV_path, stat.S_IRWXU | stat.S_IRWXG | stat.S_IRWXO)
#print("df_pivot saved")